https://www.kaggle.com/code/sukhadadharangaonkar/hotel-reviews-topic-modeling

In [47]:
# !pip install pyLDAvis==2.1.2


In [41]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# Text Preprocessing and model building
from gensim.corpora import Dictionary
import nltk
from nltk.stem import WordNetLemmatizer
import re
# Iteratively read files
import glob
import os
import pyLDAvis.gensim

# For displaying images in ipython
from IPython.display import HTML, display
from nltk.corpus import stopwords
import nltk
import pprint
from gensim.models import LdaModel



In [25]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14.0, 8.7)
#warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format


In [26]:
df = pd.read_csv("../../../DataSets/tripadvisor_hotel_reviews.csv")
print("dataset loaded...")

dataset loaded...


In [27]:
df.head()


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [28]:
df.columns


Index(['Review', 'Rating'], dtype='object')

In [29]:
reviews = df.Review.tolist()


In [30]:
wordnet_lemmatizer = WordNetLemmatizer()


In [31]:
stopwords = stopwords.words('english')


In [32]:
def preprocessText(x):
    temp = x.lower()
    temp = re.sub(r'[^\w]', ' ', temp)
    temp = nltk.word_tokenize(temp)
    temp = [wordnet_lemmatizer.lemmatize(w) for w in temp]
    temp = [word for word in temp if word not in stopwords]
    return temp


In [33]:
reviews_final = [preprocessText(review) for review in reviews]


In [34]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(reviews_final)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)


In [35]:
corpus = [dictionary.doc2bow(doc) for doc in reviews_final]


In [36]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))


Number of unique tokens: 5854
Number of documents: 20491


In [37]:
# Train LDA model.

# Set training parameters.
num_topics = 5
chunksize = 2000
passes = 10
# iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
# print(id2word)

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    #     iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


In [38]:
pprint.pprint(model.print_topics(num_words=10))
doc_lda = model[corpus]


[(0,
  '0.019*"location" + 0.016*"good" + 0.015*"breakfast" + 0.012*"clean" + '
  '0.010*"night" + 0.010*"walk" + 0.010*"nice" + 0.009*"stayed" + '
  '0.009*"helpful" + 0.008*"excellent"'),
 (1,
  '0.014*"n" + 0.012*"bed" + 0.011*"night" + 0.010*"nice" + 0.010*"floor" + '
  '0.010*"bathroom" + 0.008*"good" + 0.007*"small" + 0.007*"like" + '
  '0.007*"area"'),
 (2,
  '0.021*"service" + 0.010*"stayed" + 0.010*"wonderful" + 0.009*"time" + '
  '0.009*"best" + 0.008*"restaurant" + 0.008*"beautiful" + 0.008*"pool" + '
  '0.007*"view" + 0.007*"experience"'),
 (3,
  '0.016*"n" + 0.013*"day" + 0.008*"time" + 0.008*"got" + 0.007*"told" + '
  '0.007*"check" + 0.007*"service" + 0.007*"night" + 0.007*"hour" + '
  '0.006*"desk"'),
 (4,
  '0.022*"beach" + 0.019*"resort" + 0.015*"pool" + 0.015*"food" + 0.013*"good" '
  '+ 0.011*"time" + 0.011*"n" + 0.010*"day" + 0.009*"restaurant" + '
  '0.009*"people"')]


In [42]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary=dictionary)
vis


c:\ProgramData\Miniconda3\envs\dbsenvalt\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  return pd.DataFrame({'Term': vocab[term_ix], \


PreparedData(topic_coordinates=          x     y  topics  cluster  Freq
topic                                   
0     -0.01  0.16       1        1 28.74
4     -0.03 -0.06       2        1 24.48
3      0.12 -0.14       3        1 18.85
1      0.12  0.08       4        1 17.09
2     -0.20 -0.03       5        1 10.84, topic_info=           Term      Freq     Total Category  logprob  loglift
3588      beach  9,910.00  9,910.00  Default    30.00    30.00
3563     resort  8,475.00  8,475.00  Default    29.00    29.00
31     location 10,825.00 10,825.00  Default    28.00    28.00
285     service 10,980.00 10,980.00  Default    27.00    27.00
1037       pool  9,246.00  9,246.00  Default    26.00    26.00
...         ...       ...       ...      ...      ...      ...
409   recommend    985.42  4,790.13   Topic5    -5.30     0.64
110         day  1,247.51 13,630.78   Topic5    -5.06    -0.17
463        food  1,080.97  9,963.42   Topic5    -5.20     0.00
325    friendly  1,004.23  6,625.48   Topic5    -5.28     0.34
29         like    955.53  8,315.65   Topic5    -5.33     0.06

[437 rows x 6 columns], token_table=      Topic  Freq  Term
term                   
1528      1  0.01    00
1528      2  0.20    00
1528      3  0.67    00
1528      4  0.12    00
68        1  0.28     2
...     ...   ...   ...
810       2  0.34  year
810       3  0.11  year
810       4  0.13  year
810       5  0.31  year
987       5  0.99  yoga

[969 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 2, 3])

In [43]:
lda_corpus = model[corpus]

In [44]:
topics = []

for doc in lda_corpus:
    temp_id = []
    temp_score = []
    for doc_tuple in doc:
        temp_id.append(doc_tuple[0])
        temp_score.append(doc_tuple[1])
    index = np.argmax(temp_score)
    topics.append(temp_id[index])


In [45]:
df["Topic_num"] = topics


In [46]:
df.head(n=10)


,Review,Rating,Topic_num
0,nice hotel expensive parking got good deal sta...,4,1
1,ok nothing special charge diamond member hilto...,2,3
2,nice rooms not 4* experience hotel monaco seat...,3,1
3,"unique, great stay, wonderful time hotel monac...",5,1
4,"great stay great stay, went seahawk game aweso...",5,3
5,love monaco staff husband stayed hotel crazy w...,5,1
6,"cozy stay rainy city, husband spent 7 nights m...",5,0
7,"excellent staff, housekeeping quality hotel ch...",4,0
8,"hotel stayed hotel monaco cruise, rooms genero...",5,1
9,excellent stayed hotel monaco past w/e delight...,5,0
